<a href="https://colab.research.google.com/github/DanielAntunes-dev/ETL-Soulcode/blob/main/mdl_user.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas pyspark mysql-connector-python google-cloud-bigquery google-cloud-storage gcsfs pymongo flatten_json

In [ ]:
import pandas as pd
import os
import mysql.connector
from mysql.connector import Error
from pyspark.sql import SparkSession
from google.cloud import storage
from google.api_core.exceptions import Conflict
from google.cloud import bigquery
from google.colab import auth
from google.colab import files
from io import BytesIO
from pymongo import MongoClient

In [ ]:
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, from_unixtime
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("Processamento de CSV").getOrCreate()

arquivo = 'mdl_user.csv'

df = spark.read.csv(arquivo, header=True, inferSchema=True)

def substituir_nan(df):
    for column in df.columns:
        df = df.withColumn(column, when(col(column).isNull(), "não informado").otherwise(col(column)))
    return df

df = substituir_nan(df)

def tratar_campos_para_unix_timestamp(df, colunas):
    for coluna in colunas:
        df = df.withColumn(
            coluna,
            F.when(F.col(coluna) != 0, F.from_unixtime(F.col(coluna)))
             .otherwise(F.col(coluna))
        )
    return df

colunas_para_tratar = ["firstaccess", "lastaccess", "lastlogin", "currentlogin", "timecreated", "timemodified"]
df = tratar_campos_para_unix_timestamp(df, colunas_para_tratar)

df.show(truncate=False)

In [ ]:
import pandas as pd
from pandas_gbq import to_gbq

table_id = 'mdl_user'
dataset_id = 'dados_estudos'
project_id = 'cryptic-cache-434413-r6'

# Converter o DataFrame Spark para pandas
df_pandas = df.toPandas()

# Definir as colunas de timestamp
timestamp_columns = ["firstaccess", "lastaccess", "lastlogin", "currentlogin", "timecreated", "timemodified"]

# Formato de data
date_format = "%Y-%m-%d %H:%M:%S"

# Converter as colunas de timestamp para datetime
for col in timestamp_columns:
    if col in df_pandas.columns:
        df_pandas[col] = pd.to_datetime(df_pandas[col], format=date_format, errors='coerce')

# Converter a coluna 'id' para INTEGER
if 'id' in df_pandas.columns:
    df_pandas['id'] = pd.to_numeric(df_pandas['id'], errors='coerce').astype('Int64')

# Referência da tabela
table_ref = f'{project_id}.{dataset_id}.{table_id}'

# Carregar o DataFrame pandas para o BigQuery
to_gbq(df_pandas, destination_table=table_ref, project_id=project_id, if_exists='replace')


100%|██████████| 1/1 [00:00<00:00, 6944.21it/s]
